In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier

import os
import torch
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
 
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [16]:
historical_standings_pl = pd.read_csv('data/Premier/premier-league-standing-1993-2023.csv')
historical_standings_pl = historical_standings_pl.dropna()

print(historical_standings_pl.columns)
print(historical_standings_pl.describe())

Index(['Season_End_Year', 'Team', 'Rk', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD',
       'Pts', 'Notes'],
      dtype='object')
       Season_End_Year          Rk          MP           W           D  \
count       314.000000  314.000000  314.000000  314.000000  314.000000   
mean       2008.130573    8.993631   38.331210   16.232484    9.334395   
std           8.750654    7.206225    1.104094    7.144789    3.019579   
min        1993.000000    1.000000   38.000000    1.000000    2.000000   
25%        2001.000000    3.000000   38.000000    9.000000    7.000000   
50%        2008.000000    6.000000   38.000000   17.000000    9.000000   
75%        2016.000000   18.000000   38.000000   21.000000   11.000000   
max        2023.000000   22.000000   42.000000   32.000000   18.000000   

                L          GF          GA          GD         Pts  
count  314.000000  314.000000  314.000000  314.000000  314.000000  
mean    12.764331   56.261146   48.181529    8.079618   57.993631  
std 

In [6]:
#columna Notes no aporta.
historical_standings_pl = historical_standings_pl.drop(columns=['Notes'])
print(historical_standings_pl.columns)

Index(['Season_End_Year', 'Team', 'Rk', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD',
       'Pts'],
      dtype='object')


In [17]:
#Revisamos la cantidad y los nombres de los equipos
print(historical_standings_pl['Team'].unique())
print("\ncantidad: ", len(historical_standings_pl['Team'].unique()))

['Arsenal' 'Aston Villa' 'Crystal Palace' 'Manchester Utd' 'Middlesbrough'
 'Norwich City' "Nott'ham Forest" 'Blackburn' 'Chelsea' 'Newcastle Utd'
 'Oldham Athletic' 'Sheffield Utd' 'Swindon Town' 'Everton' 'Ipswich Town'
 'Leeds United' 'Leicester City' 'Liverpool' 'Bolton' 'Manchester City'
 'QPR' 'Sunderland' 'Barnsley' 'Charlton Ath' 'Tottenham' 'West Ham'
 'Bradford City' 'Sheffield Weds' 'Watford' 'Wimbledon' 'Coventry City'
 'Derby County' 'Fulham' 'Southampton' 'West Brom' 'Wolves'
 'Birmingham City' 'Portsmouth' 'Reading' 'Burnley' 'Hull City'
 'Blackpool' 'Stoke City' 'Swansea City' 'Wigan Athletic' 'Cardiff City'
 'Huddersfield' 'Bournemouth' 'Brighton']

cantidad:  49


In [18]:
#Cargar archivo de los partidos
matches = pd.read_csv('data/Premier/premier-league-matches-1993-2023.csv')

print(matches.columns)
print(matches.describe())

Index(['Season_End_Year', 'Wk', 'Date', 'Home', 'HomeGoals', 'AwayGoals',
       'Away', 'FTR'],
      dtype='object')
       Season_End_Year            Wk     HomeGoals     AwayGoals
count     12026.000000  12026.000000  12026.000000  12026.000000
mean       2007.713620     19.730501      1.524364      1.142525
std           9.072559     11.123916      1.306417      1.133930
min        1993.000000      1.000000      0.000000      0.000000
25%        2000.000000     10.000000      1.000000      0.000000
50%        2008.000000     20.000000      1.000000      1.000000
75%        2016.000000     29.000000      2.000000      2.000000
max        2023.000000     42.000000      9.000000      9.000000


In [19]:
#Revisan los equipos
print("equipos locales: ", matches['Home'].unique())
print("\ncantidad locales: ", len(matches['Home'].unique()))

print("\n\nequipos visitantes: ", matches['Away'].unique())
print("\ncantidad visitantes: ", len(matches['Away'].unique()))

equipos locales:  ['Coventry City' 'Leeds United' 'Sheffield Utd' 'Crystal Palace' 'Arsenal'
 'Ipswich Town' 'Everton' 'Southampton' 'Chelsea' "Nott'ham Forest"
 'Manchester City' 'Blackburn' 'Wimbledon' 'Tottenham' 'Liverpool'
 'Aston Villa' 'Oldham Athletic' 'Middlesbrough' 'Norwich City' 'QPR'
 'Manchester Utd' 'Sheffield Weds' 'Newcastle Utd' 'West Ham'
 'Swindon Town' 'Leicester City' 'Bolton' 'Sunderland' 'Derby County'
 'Barnsley' 'Charlton Ath' 'Watford' 'Bradford City' 'Fulham'
 'Birmingham City' 'West Brom' 'Portsmouth' 'Wolves' 'Wigan Athletic'
 'Reading' 'Hull City' 'Stoke City' 'Burnley' 'Blackpool' 'Swansea City'
 'Cardiff City' 'Bournemouth' 'Brighton' 'Huddersfield' 'Brentford']

cantidad locales:  50


equipos visitantes:  ['Middlesbrough' 'Wimbledon' 'Manchester Utd' 'Blackburn' 'Norwich City'
 'Aston Villa' 'Sheffield Weds' 'Tottenham' 'Oldham Athletic' 'Liverpool'
 'QPR' 'Arsenal' 'Ipswich Town' 'Coventry City' 'Sheffield Utd'
 'Leeds United' 'Crystal Palace' 'Manch

In [21]:
#Se eliminan los resultados de la temporada 2023 de las historical standings
historical_standings_pl = historical_standings_pl[historical_standings_pl['Season_End_Year'] != 2023]
